In [41]:
# Homework 4
# Center of Mass Position and Velocity
# Madison Walder

### Keep in mind this is just a template; you don't need to follow every step and feel free to change anything.
### We also strongly encourage you to try to develop your own method to solve the homework.

In [42]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl

from Readfile import Read

In [43]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy 
# and simulation snapshot
    
    def __init__(self, filename, particletype):
    # Initialize the instance of this Class with the following properties:
    
        # read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == particletype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index]
        # write your own code to complete this for positions and velocities
        self.x = self.data['x'][self.index] 
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,a,b,c,m):
    # Function to compute the center of mass position or velocity generically
    # input: array (a,b,c) of positions or velocities and the mass
    # returns: 3 floats  (the center of mass coordinates)

        # write your own code to compute the generic COM using Eq. 1 in the homework instructions
        
        #sum up all of the masses in the array to get the denominator to calculate the COM with Eq. 1
        totalmass = np.sum(m)
        
        #first define a variable to store the sum of the individual xcomp,ycomp,and zcomps and mass products
        xcompnumerator = 0.0
        ycompnumerator = 0.0
        zcompnumerator = 0.0
        
        #loop over elements in xcomponent, ycomponent, and zcomponent arrays as well as the mass array and multiply 
        #mass and xcomponents together, mass and ycomponents together, and mass and zcomponents together
        #then sum them up to get the numerator for Eq. 1 for x, y, and z components of the COM
        
        for i in range(len(a)):
            xcompindividual = a[i]*m[i]    #xcomponent and mass product for an individual particle
            xcompnumerator = xcompnumerator + xcompindividual   #sum up all individual x products to get total for numerator in Eq. 1
            
            ycompindividual = b[i]*m[i]    #ycomponent and mass product for an individual particle
            ycompnumerator = ycompnumerator + ycompindividual   #sum up all individual y products to get total for numerator in Eq. 1
            
            zcompindividual = c[i]*m[i]    #zcomponent and mass product for an individual particle
            zcompnumerator = zcompnumerator + zcompindividual   #sum up all individual z products to get total for numerator in Eq. 1

        # xcomponent Center of mass
        Acom = xcompnumerator/totalmass
        # ycomponent Center of mass
        Bcom = ycompnumerator/totalmass
        # zcomponent Center of mass
        Ccom = zcompnumerator/totalmass
        
        return Acom, Bcom, Ccom
    
    
    def COM_P(self, delta):
    # Function to specifically return the center of mass position and velocity                                         
    # input:                                                                                                           
    #        particle type (1,2,3)                                                                                     
    #        delta (tolerance)                                                                                         
    # returns: One vector, with rows indicating:                                                                                                                                                                            
    #       3D coordinates of the center of mass position (kpc)                                                             

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        XCOM, YCOM, ZCOM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        # write your own code below
        RCOM = np.sqrt(XCOM**2 + YCOM**2 + ZCOM**2)


        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        # write your own code below
        xNew = self.x - XCOM
        yNew = self.y - YCOM
        zNew = self.z - ZCOM
        RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        RMAX = max(RNEW)/2.0
        
        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        CHANGE = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while (CHANGE > delta):
            # select all particles within the reduced radius (starting from original x,y,z, m)
            # write your own code below (hints, use np.where)
            index2 = np.where(RNEW <= RMAX)
            x2 = self.x[index2]
            y2 = self.y[index2]
            z2 = self.z[index2]
            m2 = self.m[index2]

            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            # write your own code below
            XCOM2, YCOM2, ZCOM2 = self.COMdefine(x2,y2,z2,m2)
            # compute the new 3D COM position
            # write your own code below
            RCOM2 = np.sqrt(XCOM2**2 + YCOM2**2 + ZCOM2**2)

            # determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            CHANGE = np.abs(RCOM - RCOM2)
            # uncomment the following line if you want to check this                                                                                               
            #print ("CHANGE = ", CHANGE)                                                                                     

            # Before loop continues, reset : RMAX, particle separations and COM                                        

            # reduce the volume by a factor of 2 again                                                                 
            RMAX = RMAX/2.0
            # check this.                                                                                              
            #print ("maxR", RMAX)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            # write your own code below
            xNew = x2 - XCOM2
            yNew = y2 - YCOM2
            zNew = z2 - ZCOM2
            RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

            # set the center of mass positions to the refined values                                                   
            XCOM = XCOM2
            YCOM = YCOM2
            ZCOM = ZCOM2
            RCOM = RCOM2

            # create a vector to store the COM position                                                                                                                                                       
            COMP = [XCOM, YCOM, ZCOM]

        # set the correct units usint astropy and round all values
        # and then return the COM positon vector
        # write your own code below
        COMP_with_units = COMP*u.kpc   #return the position components with units of kpc
        roundedCOMP = np.around(COMP_with_units, 2)   #round the vector to 2 decimal places 
        return roundedCOMP                #return rounded position components
        
    

    def COM_V(self, COMX,COMY,COMZ):
        # Center of Mass velocity
        # input: X, Y, Z positions of the COM
        # returns 3D Vector of COM Velocities
        
        # the max distance from the center that we will use to determine the center of mass velocity                   
        RVMAX = 15.0*u.kpc

        # determine the position of all particles relative to the center of mass position
        # write your own code below
        xV = self.x*u.kpc - COMX
        yV = self.y*u.kpc - COMY
        zV = self.z*u.kpc - COMZ
        RV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        # determine the index for those particles within the max radius
        # write your own code below
        indexV = np.where(RV <= RVMAX)

        # determine the velocity and mass of those particles within the masS radius
        # write your own code below
        vxnew = self.vx[indexV]
        vynew = self.vy[indexV]
        vznew = self.vz[indexV]
        mnew = self.m[indexV]  
        
        # compute the center of mass velocity using those particles
        # write your own code below
        VXCOM, VYCOM, VZCOM = self.COMdefine(vxnew,vynew,vznew,mnew)

        #define km/s units
        kms = (u.km / u.s)
        
        COMV = [VXCOM, VYCOM, VZCOM]   # create a vector to store the COM velocity
        COMV_with_units = COMV*kms   # set the correct units using astropy
        roundedCOMV = np.around(COMV_with_units, 2)   # round all values

        # return the COM vector                                                                                        
        return roundedCOMV

In [44]:
# Create a Center of mass object for the MW, M31 and M33
# below is an example of using the class for MW
MWCOM = CenterOfMass("MW_000.txt", 2)
M31COM = CenterOfMass("M31_000.txt", 2)
M33COM = CenterOfMass("M33_000.txt", 2)

In [45]:
# Answer to question number 1
# MW:   store the position and velocity COM
MW_COMP = MWCOM.COM_P(0.1)
print("MW COM Position = ", MW_COMP)
MW_COMV = MWCOM.COM_V(MW_COMP[0], MW_COMP[1], MW_COMP[2])
print("MW COM Velocity = ", MW_COMV)

#M31: store position and velocity COM
M31_COMP = M31COM.COM_P(0.1)
print("M31 COM Position = ", M31_COMP)
M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])
print("M31 COM Velocity = ", M31_COMV)

#M33: store position and velocity COM
M33_COMP = M33COM.COM_P(0.1)
print("M33 COM Position = ", M33_COMP)
M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])
print("M33 COM Velocity = ", M33_COMV)

MW COM Position =  [-0.87  2.39 -1.42] kpc
MW COM Velocity =  [-0.47  3.41 -1.33] km / s
M31 COM Position =  [-377.66  611.43 -284.64] kpc
M31 COM Velocity =  [ 72.85 -72.14  49.  ] km / s
M33 COM Position =  [-476.22  491.44 -412.4 ] kpc
M33 COM Velocity =  [ 44.42 101.78 142.23] km / s


In [46]:
#Answer to question 2: what is the magnitude of the separation and velocity between MW and M31?
    #first determine magnitude of separation by using the distance formula
    #distance formula d = sqrt((x2-x1)^2 + (y2-y1)^2 + (z2-z1)^2)
    
#use np.subtract to subtract elements of M31 and MW then store component differences in an array
distdiff = np.subtract(M31_COMP, MW_COMP)
#print(distdiff)

#to get the magnitude of separation, square each different component and add all the squares together
#then take the square root
magofsep = np.sqrt(distdiff[0]**2 + distdiff[1]**2 + distdiff[2]**2)
print("The separation between MW and M31 in kpc =", magofsep)

#then use same concept for determining the magnitude of velocity between MW and M31
velocitydiff = np.subtract(M31_COMV, MW_COMV)

#to get the magnitude of velocity, square each different component and add all the squares together
#then take the square root
magofvelseparation = np.sqrt(velocitydiff[0]**2 + velocitydiff[1]**2 + velocitydiff[2]**2)
print("The separation between the velocities of MW and M31 in km/s =", magofvelseparation)

#comparing to the notes from Lecture 2, it seems like the calculated distance separation between MW and M31 is correct
#since in the notes it states the separation is ~ 770kpc
#Again comparing to the notes from Lecture 2, it seems like the calculated velocity difference between MW and M31 is relatively correct
#since in the notes it is ~110 km/s

The separation between MW and M31 in kpc = 770.138944671674 kpc
The separation between the velocities of MW and M31 in km/s = 116.69076141666056 km / s


In [47]:
#Answer to question 3, what is the magnitude of the current separation and velocity between M33 and M31?
#follow the same method as taken in question 2

#use np.subtract to subtract elements of M31 and M33 then store component differences in an array
dist2diff = np.subtract(M31_COMP, M33_COMP)

#to get the magnitude of separation, square each different component and add all the squares together
#then take the square root
mag2ofsep = np.sqrt(dist2diff[0]**2 + dist2diff[1]**2 + dist2diff[2]**2)
print("The separation between M31 and M33 in kpc =", mag2ofsep)

#then use same concept for determining the magnitude of velocity between MW and M31
velocity2diff = np.subtract(M31_COMV, M33_COMV)

#to get the magnitude of velocity, square each different component and add all the squares together
#then take the square root
mag2ofvelseparation = np.sqrt(velocity2diff[0]**2 + velocity2diff[1]**2 + velocity2diff[2]**2)
print("The separation between the velocities of M31 and M33 in km/s =", mag2ofvelseparation)

#comparing the velocity value calculated to that from Lecture 2, I would say that it's pretty correct because the 
#value from the lecture is 202 km/s

The separation between M31 and M33 in kpc = 201.08279712595998 kpc
The separation between the velocities of M31 and M33 in km/s = 199.36966720140757 km / s


#### Answer to question 4, why is the iterative proces to determine the COM important given that M31 and the MW are about to merge?
The iterative process to determine the COM is important since M31 and the Milky Way are supposed to merge, and the COMs for both will continue to change until they merge, where they will have the same COM position.  So using the iterative process in simulations can help us determine when they will merge with more accuracy and precision.